In [ ]:
#not used currently! old method for adding more rows about references to dataframe, but all later code ended up needing
#a different format
def add_references(df, ref_list_all):
    
    
    row_list = df[df['category'] == 'category: classification'].index
    df_class = df[df['category'] == 'category: classification']
    
    ref_dic = create_ref_dic(ref_list_all)
    
    oldest_list = []
    newest_list = []
    range_list = []
    class_dic = {}
    num_ref = []

    for element in row_list:
        ref_dic.get(element).sort()
        class_dic[element] = ref_dic.get(element) #need this otherwise the sort is forgotten
        ref_list = class_dic.get(element)
        oldest_list.append(int(ref_list[0]))
        newest_list.append(int(ref_list[len(ref_list)-1]))
        range_list.append(int(ref_list[len(ref_list)-1])- int(ref_list[0]))
        num_ref.append(len(ref_list))
   
    
    data = {'index': row_list,
            'Number of references': num_ref,
            'Oldest reference': oldest_list,
            'Newest reference' : newest_list,
            'Range of references': range_list
    } 
    
    ref = pd.DataFrame(data)
    
    df_class = df_class.reset_index()
    
    df_class = df_class.merge(ref, on = 'index')
 
    
    return df_class 

    

In [ ]:
df_2012class = add_references(df_2012, ref_list2012)

df_2021class = add_references(df_2021, ref_list2021)

In [ ]:
#method that updates the dictionary of index keys and ref list to contain tuples of ref year and age instead
#not used currently
def calc_age(dic, year):
    for key in dic.keys():
        ref_list = dic.get(key)
        with_age = []
        for ref in ref_list:
            with_age.append(year-int(ref))
        dic[key] = with_age
    return dic

In [ ]:
#looks very squished but works!
#acl code!
sns.set_context("paper", font_scale=1.5)
ax = sns.boxenplot(data=df, x='year', y='cited_age', palette=[sns.color_palette("Blues")[3]])

ax.set_xlabel("")
ax.set_ylabel("Age of citations")
plt.xticks(rotation=45)

In [ ]:
#acl code
df_max = df.groupby(['paper_id', 'year'])["cited_age"].max().reset_index()
sns.set_context("paper", font_scale=1.5)
ax = sns.boxenplot(data=df_max, x='year', y='cited_age', palette=[sns.color_palette("Blues")[3]])

ax.set_xlabel("")
ax.set_ylabel("Age of citations")
plt.xticks(rotation=45)


In [ ]:
#acl code
# normalize by total no. of *papers* in year
pfactor = df['paper_id'].groupby(df['year']).nunique('paper_id').reset_index()
counts['count'] = counts['count'].astype(float)
pcounts = counts.apply(lambda x: [x[0], x[1], x[2] / float(pfactor[pfactor['year']==x[0]]['paper_id']), x[3]], axis=1, result_type='broadcast')

# cumulative count of citations of age X or older
# need to reverse first so we start from the oldest...
pcounts = pcounts.sort_values(by='cited_age', ascending=False)
# then cumulatively sum up, for each year individually
pcounts['cumcount'] = pcounts.groupby('year')['count'].transform(pd.Series.cumsum)

sns.lineplot(data=pcounts, x='cited_age', y='cumcount', hue='year', legend='full')


In [ ]:
#modified code from acl repo! Used to calculate/show the count of ages per year, looking for maxima
#in final version removed all the commented out lines:

# Count citations by year & cited_age
#counts = df2012_with_year.groupby(['year', 'cited_age']).count()
counts = df2021_with_year.groupby(['year', 'cited_age']).count()
counts = counts.reset_index().filter(items=['year', 'cited_age', 'venue'])
counts.columns.values[-1] = "count"
#counts[counts['count'] == counts['count'].max()]

# Normalize by total no. of citations in year
#nfactor = df.groupby(['year']).count().reset_index()
#counts['count'] = counts['count'].astype(float)
#counts['percent_of_citations'] = counts.apply(lambda x: 100 * x[2] / float(nfactor[nfactor['year']==x[0]]['paper_id']), axis=1)


test = counts.sort_values(by = ['cited_age'], ascending = True)
test.head(50)

#sns.lineplot(data=counts, x='cited_age', y='count', hue='year', legend='full')

#plt.savefig('percent of citations vs age of citations')
#plt.show()


In [ ]:
#acl code - can definitely see that that one paper from 1872(?) is insanely skewing things
#wait no, this makes no sense? Not sure this is correct at all?

#We can sanity-check that this calculation is correct by directly calculating the average number of citations per paper:

# no. of citations per paper
cpp = df.groupby(['paper_id', 'year']).count()['cited_age'].reset_index()
# mean by year
cpp.groupby('year').mean()